In [ ]:
def print_statistics(array): 
  import scipy.stats as scs

  sta = scs.describe(array) 
  print("%s %s" % ('statistic' , 'value'))
  print(30 * "-")
  print("%14s %15.5f" % ('size', sta[0]))
  print("%14s %15.5f" % ('min', sta[1][0]))
  print("%14s %15.5f" % ('max', sta[1][1]))
  print("%14s %15.5f" % ('mean', sta[2]))
  print("%14s %15.5f" % ('std', np.sqrt(sta[3])))
  print("%14s %15.5f" % ('skew', sta[4]))
  print("%14s %15.5f" % ('kurtosis', sta[5]))


In [ ]:
def normality_tests(arr): 
  import scipy.stats as scs

  print("Skew of data set %14.3f" % scs.skew(arr))
  print("Skew test p-value %14.3f" % scs.skewtest(arr)[1])
  print("Kurt of data set %14.3f" % scs.kurtosis(arr))
  print("Kurt test p-value %14.3f" % scs.kurtosistest(arr)[1])
  print("Norm test p-value %14.3f" % scs.normaltest(arr)[1])

In [ ]:
def plot_hist(log_returns, r, sigma, M):
  import scipy.stats as scs
  from numpy import sqrt
  import matplotlib.pyplot as plt
    
  plt.hist(log_returns, bins=70, normed=True, label='frequency') 
  plt.grid (True) 
  plt.xlabel('log-return') 
  plt.ylabel('frequency') 
  x = np.linspace(plt.axis()[0], plt.axis()[1]) 
  plt.plot(x, scs.norm.pdf(x, loc = r/M, scale = sigma/sqrt(M)), 'r', lw = 2.0, label = 'pdf') 
  plt.legend()

In [ ]:
def gen_mc_paths(S0, r, sigma, T, M=50, I=250000):
  from numpy import random, zeros, exp, sqrt, float64

  dt = float(T)/M 
  paths = zeros((M + 1, I), float64) 
  paths[0] = S0 
  for t in range(1, M + 1): 
    rand = random.standard_normal(I) 
    rand = (rand - rand.mean()) / rand.std() 
    paths[t] = paths[t - 1] * exp((r - 0.5 * sigma ** 2) * dt + sigma * sqrt(dt) * rand)
  return paths

In [ ]:
def reference_mc_data_test():
  import numpy as np

  print("Normal distributed MC simulated data")
  print(33 * "-")
  mc_paths = gen_mc_paths(100., 0.05, 0.2, 1.0)
  mc_log_returns = np.log(mc_paths[1:]/mc_paths[0:-1])
  normality_tests(mc_log_returns.flatten())
  plot_hist(mc_log_returns.flatten(), 0.05, 0.2, 50)

In [ ]:
def stock_index_test():
  import pandas as pd
  from dao import InvestDao

  dao = InvestDao(host="172.17.0.1", port=3306, user="user", passwd="passwd", db="db")
  rows = dao.sql("SELECT close FROM stock_quotes WHERE dtyymmdd > '2016-06-01' AND ticker = 'WIG20'")
  data = pd.DataFrame()
  data['WIG20'] = [float(r[0]) for r in rows]
  log_returns = np.log(data / data.shift(1))  
  log_data = np.array(log_returns['WIG20'].dropna())
  
  print("WIG20 index data")
  print(33 * "-")
  normality_tests(log_data)  
  log_returns.hist(bins = 50, figsize = (9, 6))

In [ ]:
reference_mc_data_test()

In [ ]:
stock_index_test()